In [ ]:
# 1
threshold가 0.3으로 가입이 쉬워진다면 실제로는 자격이 없지만 가입될 수 있는 경우가 늘어나기 때문에
다시 말해 FP가 커지기 때문에 Type 1 Error가 늘어납니다.

In [ ]:
# 2 - 1
Accuracy : 정확도 , 모델의 예측이 성공한 비율 -> (TP+TN) / (TP+TN+FP+FN)
Precision : 정밀도, 모델이 True라고 예측한 값들 중 실제로 True인 경우의 비율 -> (TP) / (TP + FP)
Recall : 재현율, 실제 True인 것 중에서 모델이 True라고 예측한 것의 비율 -> (TP) / (TP + FN)

In [ ]:
# 2 - 2
범죄에 대한 판결의 경우를 생각한다면 실제 경우는 죄가 있음, 죄가 없음, 예측은 판결이 되어 유죄판결, 무죄판결
Precision은 유죄판결은 내린 경우 중 실제로 죄가 있는 비율
Recall은 실제로 죄가 있는 경우 중 유죄판결을 받은 비율

이 경우 threshold를 높인다면 유죄를 판결받을 확률이 낮아집니다. 그렇다면 1종의 오류인 FP가 낮아지고 FN은 높아지는데 
바꿔 생각해보면 죄가 있는 사람에게 무죄판결을 하는 경우는 많아지고 죄가 없는 사람에게 유죄판결을 하는 경우는 적어지는데
많은 범죄자를 놓치더라도 한 명의 억울한 판결을 받는 경우를 막을 수 있기 때문에 합리적이라고 생각합니다.

In [1]:
# 3
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable 

In [2]:
# MNIST Dataset (Images and Labels) 
train_dataset = dsets.MNIST(root ='./data',  
                            train = True,  
                            transform = transforms.ToTensor(), 
                            download = True) 
  
test_dataset = dsets.MNIST(root ='./data',  
                           train = False,  
                           transform = transforms.ToTensor()) 

In [3]:
# Hyper Parameters  
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# Dataset Loader (Input Pipline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,  
                                           batch_size = batch_size,  
                                           shuffle = True) 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,  
                                          batch_size = batch_size,  
                                          shuffle = False) 

In [5]:
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) 
  
    def forward(self, x): 
        out = self.linear(x) 
        return out 

In [6]:
# our model
model = LogisticRegression(input_size, num_classes) 

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [8]:
# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f'
                 % (epoch + 1, num_epochs, i+1,
                   len(train_dataset) // batch_size, loss.data))
            

Epoch: [1/5], Step: [100/600], Loss: 2.2297
Epoch: [1/5], Step: [200/600], Loss: 2.1092
Epoch: [1/5], Step: [300/600], Loss: 2.0007
Epoch: [1/5], Step: [400/600], Loss: 1.9391
Epoch: [1/5], Step: [500/600], Loss: 1.8162
Epoch: [1/5], Step: [600/600], Loss: 1.7962
Epoch: [2/5], Step: [100/600], Loss: 1.6750
Epoch: [2/5], Step: [200/600], Loss: 1.6059
Epoch: [2/5], Step: [300/600], Loss: 1.6214
Epoch: [2/5], Step: [400/600], Loss: 1.6023
Epoch: [2/5], Step: [500/600], Loss: 1.5166
Epoch: [2/5], Step: [600/600], Loss: 1.4229
Epoch: [3/5], Step: [100/600], Loss: 1.4924
Epoch: [3/5], Step: [200/600], Loss: 1.4630
Epoch: [3/5], Step: [300/600], Loss: 1.2327
Epoch: [3/5], Step: [400/600], Loss: 1.3669
Epoch: [3/5], Step: [500/600], Loss: 1.2947
Epoch: [3/5], Step: [600/600], Loss: 1.2579
Epoch: [4/5], Step: [100/600], Loss: 1.2831
Epoch: [4/5], Step: [200/600], Loss: 1.2135
Epoch: [4/5], Step: [300/600], Loss: 1.1582
Epoch: [4/5], Step: [400/600], Loss: 1.1164
Epoch: [4/5], Step: [500/600], L

In [10]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28 * 28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the model on the 10000 test images: %d %%' %(
       100* correct / total ))

Accuracy of the model on the 10000 test images: 82 %


In [11]:
# 4-1
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [12]:
# For reproducibility
torch.manual_seed(1)

In [13]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: 0.578, b: -0.413 Cost: 2.147149
Epoch  100/1000 W: 1.066, b: -0.150 Cost: 0.003239
Epoch  200/1000 W: 1.052, b: -0.118 Cost: 0.002002
Epoch  300/1000 W: 1.041, b: -0.093 Cost: 0.001237
Epoch  400/1000 W: 1.032, b: -0.073 Cost: 0.000764
Epoch  500/1000 W: 1.025, b: -0.057 Cost: 0.000472
Epoch  600/1000 W: 1.020, b: -0.045 Cost: 0.000292
Epoch  700/1000 W: 1.016, b: -0.035 Cost: 0.000180
Epoch  800/1000 W: 1.012, b: -0.028 Cost: 0.000111
Epoch  900/1000 W: 1.010, b: -0.022 Cost: 0.000069
Epoch 1000/1000 W: 1.008, b: -0.017 Cost: 0.000043


In [ ]:
# 이 데이터는 전체 양이 그리 많지 않기 때문에 SGD 대신 GD를 사용해도 크게 문제가 없을 것 같습니다.
# 미분에 대한 함수식을 짭니다. 미분 => lim {f(x+h)-f(x)}/h (h->0)이니
# h에 매우 작은 값을 집어 넣는다면 근사값을 구할 수 있을 것입니다.
# Loss function을 W와 b를 이용하여 다시 짠 다음 편미분을 통해 계속해서 gradient를 갱신하여 
# 우리가 아는 식의 방법으로 weight를 계속해서 update하면 될 것 같습니다.

In [ ]:
# 4-2

In [33]:
# For reproducibility
torch.manual_seed(1)

In [34]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정 -> Adam 알고리즘 사용 -> SGD와 달리 1보다 작은 쪽에서 점점 1에 가까워짐
optimizer = optim.Adam(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: 0.525, b: -0.431 Cost: 2.147149
Epoch  100/1000 W: 0.992, b: 0.032 Cost: 0.000333
Epoch  200/1000 W: 0.990, b: 0.022 Cost: 0.000072
Epoch  300/1000 W: 0.993, b: 0.017 Cost: 0.000040
Epoch  400/1000 W: 0.995, b: 0.012 Cost: 0.000020
Epoch  500/1000 W: 0.997, b: 0.008 Cost: 0.000008
Epoch  600/1000 W: 0.998, b: 0.005 Cost: 0.000003
Epoch  700/1000 W: 0.999, b: 0.003 Cost: 0.000001
Epoch  800/1000 W: 0.999, b: 0.002 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: 0.001 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: 0.000 Cost: 0.000000


In [35]:
# For reproducibility
torch.manual_seed(1)

In [36]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정 -> RMSprop 알고리즘 사용 -> 1로 미분값이 0이어도 계속 구하면서 이동함
optimizer = optim.RMSprop(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: 0.615, b: -0.341 Cost: 2.147149
Epoch  100/1000 W: 0.992, b: 0.019 Cost: 0.000052
Epoch  200/1000 W: 0.996, b: 0.008 Cost: 0.000010
Epoch  300/1000 W: 0.999, b: 0.002 Cost: 0.000001
Epoch  400/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  500/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  700/1000 W: 0.984, b: -0.016 Cost: 0.002620
Epoch  800/1000 W: 0.999, b: -0.001 Cost: 0.000006
Epoch  900/1000 W: 0.998, b: -0.002 Cost: 0.000030
Epoch 1000/1000 W: 0.996, b: -0.004 Cost: 0.000109


In [37]:
# For reproducibility
torch.manual_seed(1)

In [38]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정 -> Momentum 추가 -> momentum이 크다 ex)1이면 크게 이동해서 잘 못찾았음, 반면 작으면 ex)0.01은 없는것과(0과) 비슷
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # 보통 0.9사용

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: 0.578, b: -0.413 Cost: 2.147149
Epoch  100/1000 W: 1.007, b: -0.008 Cost: 0.000053
Epoch  200/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  300/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  400/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  500/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  700/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  800/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: -0.000 Cost: 0.000000
